# LAB 02.03 - Pandas

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/fagonzalezo/ai4eng-unal/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()


In [2]:
from local.lib.rlxmoocapi import submit, session
session.LoginSequence(endpoint=init.endpoint, course_id=init.course_id, lab_id="L02.03", varname="student");

In [1]:
import numpy as np
import pandas as pd

## Task 1: Extract data

we have a dataframe of items with a price and, **sometimes**, with an extra column (`margin`).

You will have to complete a function that will **filter** the dataframe selecting the rows:

- whose price is > 100
- **OR** whose margin is >10, if the column `margin` is present

your function must return **A LIST** with the item ids of the selected rows

For instance, with the following dataframe

             price  category
    itemid                  
    56556   108.15       2.0
    73065    83.54       2.0
    36619   114.42       0.0
    73414    82.94       0.0
    13410   115.13       2.0
    66153    91.50       1.0
    77380    85.82       0.0
    73249    95.95       0.0
    11654   100.22       2.0
    11972    77.16       1.0

your must return the following list:

     [56556, 36619, 13410, 11654]
     
But if you get the following dataframe

             price  category  margin
    itemid                          
    39059    98.11       0.0   11.04
    19526    98.11       1.0   11.25
    78176    94.34       1.0   10.51
    50948   102.37       1.0   10.77
    12111    98.07       1.0    8.50
    56191    98.53       1.0   11.65
    38887    91.49       2.0   11.24
    77915   117.30       0.0    8.64
    55010    96.13       0.0    8.95
    45925    98.59       1.0   10.45
    
you must return the following list

    [39059, 19526, 78176, 50948, 56191, 38887, 77915, 45925]

In [2]:
def create_df(missing=False, n=10):
    itemid   = np.random.randint(100000, size=n)+1000
    category = np.random.randint(3, size=n)
    price    = np.round(np.random.normal(loc=100, scale=10, size=n),2)
    margin   = np.round(np.random.normal(loc=10, scale=1, size=n),2)
    
    if missing:
        nmissing = np.random.randint(len(price)//2)+2                                     
        price[np.random.permutation(len(price))[:nmissing]] = np.nan
    
    d = pd.DataFrame(np.r_[[price, category, margin]].T, index=itemid, columns=["price", "category", "margin"])
    d.index.name="itemid"
    if np.random.random()>.5:
        d = d[d.columns[:2]]
        
    return d

In [3]:
d = create_df()
d

,price,category,margin
itemid,,,
51235,104.25,2.0,10.52
30457,105.27,1.0,10.64
47451,104.87,2.0,10.25
37397,105.21,0.0,10.24
14890,91.28,1.0,10.23
69690,106.99,2.0,8.46
11045,72.04,0.0,9.91
64913,96.09,1.0,12.42
8998,96.06,0.0,11.28


In [ ]:
def select_items(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    
    if df.columns.__contains__('margin'): df = df[(df['price'] > 100) | (df['margin'] > 10)]
    else: df = df[df['price'] > 100]
    
    result = list(df.index)
    
    return result

**manually check your answer**

In [11]:
d = create_df()
d

,price,category
itemid,,
51944,88.93,2.0
50368,97.25,2.0
65462,85.05,0.0
52492,85.11,1.0
42922,108.22,0.0
75936,104.66,1.0
76040,89.22,1.0
29897,116.16,0.0
19548,91.71,2.0


In [12]:
select_items(d)

[42922, 75936, 29897]

**submit your code**

In [12]:
student.submit_task(globals(), task_id="task_01");

## Task 2: Group statistics

Complete the following function so that it returns a dataframe with the average, max and min **prices** per category. 

For instance, for the following dataframe

             price  category  margin
    itemid                          
    17946    93.85       1.0   10.64
    61190    91.72       1.0    9.76
    39639   100.16       1.0   10.67
    17791   110.44       2.0    9.65
    7333    101.05       1.0    9.69
    77362   122.33       0.0   11.14
    92646   108.13       2.0   10.58
    27797    85.52       2.0   10.88
    31746    97.56       0.0    9.75
    12355   101.04       2.0    9.51
    
you should return the following dataframe

                 media  maximo  minimo
    categoria                         
    0         109.9450  122.33  97.56
    1          96.6950  101.05  91.72
    2         101.2825  110.44  85.52
    
observe that your result
- must not be a **multilevel** columnset.
- the column names and the index name must be **exactly** as in the example.
- the **index** must be of type **int**.

In [13]:
d.describe()

,price,category
count,10.000000,10.000000
mean,96.065000,0.900000
std,10.476839,0.875595
min,85.050000,0.000000
25%,89.002500,0.000000
50%,93.025000,1.000000
75%,102.807500,1.750000
max,116.160000,2.000000


In [25]:
def get_stats(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    df['category'] = df['category'].astype(int)
    result = df.groupby('category').agg(
                mean=('price', 'mean'),
                min=('price', 'min'),
                max=('price', 'max')
            )
    return result

**manually check your answer**

In [23]:
d = create_df()
d

,price,category,margin
itemid,,,
8182,112.20,0.0,9.99
88397,106.14,2.0,12.52
2468,92.77,2.0,9.46
1990,86.96,2.0,9.39
66025,89.52,1.0,11.77
84329,107.86,2.0,10.49
19128,86.16,1.0,10.08
76138,98.54,1.0,10.91
70826,92.57,0.0,11.15


In [26]:
get_stats(d)

,mean,min,max
category,,,
0,102.385000,92.57,112.20
1,91.406667,86.16,98.54
2,99.036000,86.96,107.86


**submit your code**

In [16]:
student.submit_task(globals(), task_id="task_02");

## Task 3: Fill in missing data

Fill in the missing data in the **price** column with the following procedure:

- compute the mean and std of the avaialable prices
- sample from a normal distribution with the computed mean and std (see [`np.random.normal`])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
- substitute the missing values with the samples

For instance, for this input dataframe:

             price  category  margin
    itemid                          
    18922      NaN       1.0   10.32
    69500   121.25       1.0   10.22
    76442    90.25       1.0   12.60
    33863   106.51       0.0   10.26
    15904    95.87       1.0   11.51
    41946   103.47       2.0    9.85
    85451    93.08       2.0    9.56
    70028   116.68       1.0    9.11
    26860      NaN       2.0    9.71
    12807    91.48       0.0    9.77
    
your solution might be similar to this (not exactly the same as you will be sampling data):


                 price  category  margin
    itemid                              
    18922    97.441188       1.0   10.32
    69500   121.250000       1.0   10.22
    76442    90.250000       1.0   12.60
    33863   106.510000       0.0   10.26
    15904    95.870000       1.0   11.51
    41946   103.470000       2.0    9.85
    85451    93.080000       2.0    9.56
    70028   116.680000       1.0    9.11
    26860   103.294843       2.0    9.71
    12807    91.480000       0.0    9.77
    
    
**WARN**: your function must not modify the original dataframe, make a copy of the input dataframe, fill the values in the copy and return it.

**HINT**: use the [`isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) method of a dataframe or a series.

In [7]:
d = create_df(missing=True)
d

In [38]:
def fillna(df):
    # make sure to make a copy in case you modify the original df
    df = df.copy()
    miu, sigma = df['price'].mean() , df['price'].std()
    n = df['price'].isna().sum()
    df.loc[df['price'].isna(),'price'] = np.random.normal(miu, sigma, n)
    result = df
    return result

**check manually your code**

In [36]:
d = create_df(missing=True)
d

,price,category,margin
itemid,,,
57718,92.66,1.0,10.61
79054,NaN,2.0,8.62
17567,NaN,1.0,8.58
10426,NaN,2.0,9.46
70648,NaN,0.0,8.91
83214,91.43,0.0,9.76
13568,99.64,2.0,11.04
72730,NaN,2.0,12.12
10408,90.88,0.0,10.64


In [39]:
fillna(d)

,price,category,margin
itemid,,,
57718,92.660000,1.0,10.61
79054,95.410243,2.0,8.62
17567,90.516296,1.0,8.58
10426,99.131838,2.0,9.46
70648,92.506107,0.0,8.91
83214,91.430000,0.0,9.76
13568,99.640000,2.0,11.04
72730,96.809623,2.0,12.12
10408,90.880000,0.0,10.64


**submit your code**

In [24]:
student.submit_task(globals(), task_id="task_03");